# 🎯 Pumpfun Kaggle — Plan & Starter Code (CatBoost Pipeline)

Ce document te donne un **plan d’attaque clair** et un **début de code** prêt à adapter pour le challenge *Alpha Radar: Solana Sprint* (Pumpfun/Solana), avec **CatBoost**, **feature engineering limité aux 30 premières secondes**, **optimisation du Jaccard** et **contrainte Recall ≥ 75%**.

---

## 1) Plan d’attaque (haut niveau)

1. **Ingestion & contraintes**

   * Charger les CSV de transactions (30s) et la liste des cibles (target tokens).
   * **Clé**: `mint_token_id`. Fenêtre d’analyse **0–30 s** post-mint.
   * **Soumission**: exactement **64 208** lignes (maj oct. 2025), binaire `{0,1}`.

2. **Vérifs de base**

   * Types/dtypes, valeurs manquantes, domaines (`trade_mode` ∈ {buy, sell, other}).
   * Distribution de `is_target` (déséquilibre attendu).

3. **Feature engineering (par token, dans 0–30 s)**

   * **Activité/flux**: `buy_count`, `sell_count`, `total_count`, `buy_sell_ratio`.
   * **Volumes**: `token_volume`, `sol_volume`, `liquidity_ratio`, `market_cap_usd` (stats: sum/mean/max/std, CV).
   * **Prix/techniques**: `relative_strength_index`, `bollinger_relative_position`, `volume_oscillator`, `rate_of_change`, `money_flow_index` (mean/max/std, trending: dernière valeur vs moyenne).
   * **Réserves pool**: `virtual_sol_reserves`, `virtual_token_reserves` (mean/max, ratio).
   * **Comportements acteurs**: `total_holders`, `current_holders`, `holder_ratio`, diversité acheteurs (approx via `current_holders/total_holders`).
   * **Créateur**: `creator_fee`, `creator_fee_pump`, `creator_balance`, `creator_sold`, indicateurs `creator_activity` (a vendu ? a un gros solde ? ratio `creator_sold/token_volume`).
   * **Gas/Frais**: `consumed_gas` (mean/sum/max), `fee` (mean/max), proxy d’urgence/network friction.
   * **Deltas**: `token_delta`, `sol_delta` (sum/sign, ratios au volume).
   * **Temporalité intra-30s**: features **early vs late** (ex: stats sur [0–10s], [10–20s], [20–30s]) si le dataset contient un horodatage assez fin.

4. **Agrégation → 1 ligne/token**

   * `groupby(mint_token_id)` pour produire la matrice finale X (une ligne par token).

5. **Labellisation**

   * Joindre la liste `target_tokens` → `is_target ∈ {0,1}`.

6. **Split & Validation**

   * Split **stratifié par `is_target`** (ex: 80/20). Option *time-aware* si on veut éviter fuite temporelle (train sur début du mois, val sur fin).

7. **CatBoost**

   * Gestion déséquilibre: `class_weights` ou `scale_pos_weight`.
   * `eval_metric='Logloss'` + `use_best_model=True` (early stopping).
   * Seed fixe pour reproductibilité.

8. **Seuil optimisé Jaccard (avec contrainte Recall ≥ 0.75)**

   * Balayer `threshold ∈ [0,1]`.
   * Pour chaque seuil: calculer `TP, FP, FN`, **Recall** et **Jaccard**.
   * Choisir le **seuil max Jaccard** **sous la contrainte** `Recall ≥ 0.75`.

9. **Évaluation & Rapport**

   * Afficher **Jaccard**, **Recall**, **Précision**, **F1**.
   * Matrice de confusion + "**confusion Jaccard**" (TP/(TP+FP+FN)).

10. **Soumission**

    * Prédire sur l’**éval set** → `mint_token_id,is_target` (binaire via seuil choisi).
    * **Vérifier**: exactement **64 208** lignes + **header**.

---

In [4]:
# pumpfun_pipeline.py


import os
import math
import json
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Tuple, Dict, List
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


# CatBoost
from catboost import CatBoostClassifier, Pool


# -----------------------------
# Config
# -----------------------------
@dataclass
class Config:
    transactions_csv: str = "C:\\Users\\tje6d8f\\OneDrive - Colruyt Group NV\\Bureau\\Pers\\Alpha radar pumpfun 30sec\\Fichiers test\\september_2025_first30s_chunk_001" # à adapter
    targets_csv: str = "C:\\Users\\tje6d8f\\OneDrive - Colruyt Group NV\\Bureau\\Pers\\Alpha radar pumpfun 30sec\\Fichier validation\\Validation" # à adapter
    output_dir: str = "./outputs"
    random_state: int = 42
    test_size: float = 0.2
    recall_constraint: float = 0.75
    expected_eval_rows: int = 64208 # maj Oct 2025


CFG = Config()


os.makedirs(CFG.output_dir, exist_ok=True)

ModuleNotFoundError: No module named 'catboost'

In [ ]:
# -----------------------------
# Utils métriques
# -----------------------------

def jaccard_score(tp: int, fp: int, fn: int) -> float:
    denom = tp + fp + fn
    return tp / denom if denom > 0 else 0.0


def evaluate_threshold(y_true: np.ndarray, y_prob: np.ndarray, thr: float) -> Dict[str, float]:
    y_pred = (y_prob >= thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    f1 = 2*precision*recall/(precision+recall) if (precision+recall) > 0 else 0.0
    jacc = jaccard_score(tp, fp, fn)
    return {
        "threshold": thr,
        "tp": tp, "fp": fp, "fn": fn, "tn": tn,
        "recall": recall, "precision": precision, "f1": f1, "jaccard": jacc
    }


def best_threshold_under_recall(y_true: np.ndarray, y_prob: np.ndarray, min_recall: float = 0.75) -> Dict[str, float]:
    # Balayage fin autour de la zone utile
    candidates = np.linspace(0.01, 0.99, 99)
    reports = [evaluate_threshold(y_true, y_prob, t) for t in candidates]
    feasible = [r for r in reports if r["recall"] >= min_recall]
    if not feasible:
        # pas de seuil qui respecte la contrainte, on prend celui qui maximise le recall (fallback)
        return max(reports, key=lambda r: r["recall"])
    return max(feasible, key=lambda r: r["jaccard"])  # maximise Jaccard sous contrainte de recall



In [ ]:
# -----------------------------
# Chargement & dtypes
# -----------------------------

def dtype_map() -> Dict[str, str]:
    return {
        "index": "int32",
        "timestamp": "string",  # parsé ensuite en datetime si besoin
        "mint_token_id": "string",
        "holder": "string",
        "trade_mode": "category",
        "token_quantity": "float32",
        "creator": "string",
        "creator_fee": "float32",
        "creator_fee_pump": "float32",
        "market_cap_usd": "float32",
        "token_delta": "float32",
        "sol_delta": "float32",
        "buy_count": "int16",
        "sell_count": "int16",
        "total_count": "int16",
        "token_volume": "float32",
        "sol_volume": "float32",
        "liquidity_ratio": "float32",
        "virtual_sol_reserves": "float32",
        "virtual_token_reserves": "float32",
        "consumed_gas": "float32",
        "fee": "float32",
        "relative_strength_index": "float32",
        "bollinger_relative_position": "float32",
        "volume_oscillator": "float32",
        "rate_of_change": "float32",
        "money_flow_index": "float32",
        "total_holders": "int32",
        "current_holders": "int32",
        "top10_percent_total": "float32",
        "creator_balance": "float32",
        "creator_sold": "float32",
        "holder_ratio": "float32",
        "buy_sell_ratio": "float32",
    }


def load_transactions(path: str) -> pd.DataFrame:
    usecols = list(dtype_map().keys())
    df = pd.read_csv(path, usecols=usecols, dtype=dtype_map())
    # Optionnel: df['timestamp'] = pd.to_datetime(df['timestamp'], utc=True, errors='coerce')
    return df


def load_targets(path: str) -> pd.DataFrame:
    # Suppose un CSV avec une colonne 'mint_token_id' listant les tokens cibles
    t = pd.read_csv(path)
    # Harmonisation colonnes
    col = None
    for c in t.columns:
        if c.lower() in {"mint_token_id", "token", "mint", "mint_id"}:
            col = c
            break
    if col is None:
        raise ValueError("Impossible de trouver la colonne mint_token_id dans le fichier targets.")
    t = t[[col]].rename(columns={col: "mint_token_id"})
    t["is_target"] = 1
    return t



In [ ]:
# -----------------------------
# Feature engineering (agg par token)
# -----------------------------

def _agg_stats(series: pd.Series, prefix: str) -> Dict[str, float]:
    return {
        f"{prefix}_sum": series.sum(),
        f"{prefix}_mean": series.mean(),
        f"{prefix}_max": series.max(),
        f"{prefix}_std": series.std(ddof=0),
    }


def build_features(df: pd.DataFrame) -> pd.DataFrame:
    # Séparation par trade_mode pour quelques compteurs
    df["is_buy"] = (df["trade_mode"].astype("string") == "buy").astype("int8")
    df["is_sell"] = (df["trade_mode"].astype("string") == "sell").astype("int8")

    groups = []
    for token, g in df.groupby("mint_token_id", sort=False):
        feats = {"mint_token_id": token}

        # Comptes bruts (existants mais on les recalcule en sécurité)
        feats.update({
            "tx_count": int(g.shape[0]),
            "buy_count_agg": int(g["is_buy"].sum()),
            "sell_count_agg": int(g["is_sell"].sum()),
        })
        feats["buy_sell_ratio_agg"] = (feats["buy_count_agg"] / max(1, feats["sell_count_agg"]))

        # Volumes & deltas
        for col in [
            "token_volume","sol_volume","market_cap_usd","liquidity_ratio",
            "token_delta","sol_delta","consumed_gas","fee",
            "virtual_sol_reserves","virtual_token_reserves",
            "relative_strength_index","bollinger_relative_position","volume_oscillator",
            "rate_of_change","money_flow_index",
        ]:
            feats.update(_agg_stats(g[col].astype("float32"), col))

        # Holders & créateur
        for col in [
            "total_holders","current_holders","top10_percent_total",
            "creator_fee","creator_fee_pump","creator_balance","creator_sold",
            "holder_ratio",
        ]:
            # la plupart sont quasi constants sur 30s → mean suffit
            feats[f"{col}_mean"] = g[col].astype("float32").mean()
            feats[f"{col}_max"] = g[col].astype("float32").max()

        # Indicateurs créateur
        feats["creator_sold_flag"] = 1 if (g["creator_sold"].max() > 0) else 0
        vol = max(1e-6, g["token_volume"].sum())
        feats["creator_sell_volume_ratio"] = float(g["creator_sold"].sum() / vol)

        # Diversité acheteurs proxy
        th = float(g["total_holders"].max()) if not g["total_holders"].isna().all() else 0.0
        ch = float(g["current_holders"].max()) if not g["current_holders"].isna().all() else 0.0
        feats["holder_diversity_ratio_max"] = (ch / th) if th > 0 else 0.0

        groups.append(feats)

    feats_df = pd.DataFrame(groups)
    feats_df = feats_df.replace([np.inf, -np.inf], np.nan).fillna(0)
    return feats_df



In [ ]:
# -----------------------------
# Train / Val
# -----------------------------

def train_catboost(X: pd.DataFrame, y: pd.Series, cfg: Config = CFG) -> Tuple[CatBoostClassifier, Dict[str, float]]:
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=cfg.test_size, stratify=y, random_state=cfg.random_state
    )

    train_pool = Pool(X_train, y_train)
    val_pool = Pool(X_val, y_val)

    # Gestion du déséquilibre: poids inverses de fréquence
    pos_weight = (len(y_train) - y_train.sum()) / max(1, y_train.sum())

    model = CatBoostClassifier(
        iterations=2000,
        depth=6,
        learning_rate=0.03,
        loss_function='Logloss',
        eval_metric='Logloss',
        random_seed=cfg.random_state,
        class_weights=[1.0, float(pos_weight)],
        verbose=200,
        use_best_model=True,
        od_type='Iter',
        od_wait=200,
    )

    model.fit(train_pool, eval_set=val_pool)

    # Optimisation du seuil (Jaccard sous contrainte Recall)
    y_val_prob = model.predict_proba(X_val)[:,1]
    report = best_threshold_under_recall(y_val, y_val_prob, cfg.recall_constraint)
    return model, report



In [ ]:
# -----------------------------
# Pipeline principal
# -----------------------------

def main():
    print("[1/6] Chargement des données…")
    df = load_transactions(CFG.transactions_csv)
    targets = load_targets(CFG.targets_csv)

    print("[2/6] Feature engineering (agg par token)…")
    feats = build_features(df)

    print("[3/6] Jointure labels…")
    data = feats.merge(targets, on="mint_token_id", how="left")
    data["is_target"] = data["is_target"].fillna(0).astype(int)

    # Séparation X/y
    y = data["is_target"].astype(int)
    X = data.drop(columns=["is_target", "mint_token_id"])  # garder id à part pour soumission

    print("[4/6] Entraînement CatBoost…")
    model, thr_report = train_catboost(X, y)

    print("Seuil choisi (contrainte recall ≥ {0:.2f}):".format(CFG.recall_constraint))
    print(json.dumps(thr_report, indent=2))

    # Sauvegarde du modèle
    model_path = os.path.join(CFG.output_dir, "catboost_model.cbm")
    model.save_model(model_path)
    print(f"Modèle sauvegardé: {model_path}")

    # Exemple de génération soumission (à adapter à l'eval set exact de 64,208 rows)
    print("[5/6] Génération soumission…")
    ids = feats[["mint_token_id"]].copy()
    probs = model.predict_proba(X)[:,1]
    y_pred_bin = (probs >= thr_report["threshold"]).astype(int)

    submit = ids.copy()
    submit["is_target"] = y_pred_bin

    # Vérif de la contrainte de lignes si on est sur l'eval set officiel
    if len(submit) != CFG.expected_eval_rows:
        print(f"[WARN] Lignes soumission = {len(submit)} ≠ {CFG.expected_eval_rows}. Vérifie que tu utilises l'eval set.")

    sub_path = os.path.join(CFG.output_dir, "submission.csv")
    submit.to_csv(sub_path, index=False)
    print(f"Soumission écrite: {sub_path}")

    # Rapport simple
    print("[6/6] Rapport validation (local split)…")
    # Re-calcul sur le split val déjà fait dans train_catboost si besoin
    # Ici on peut re-écouter la perf globale train+val pour un ordre de grandeur
    # (la perf Kaggle dépendra de l'eval set et de la non-fuite de données)

if __name__ == "__main__":
    main()


---

## 3) Pistes d’amélioration (prochaines itérations)

* **Temporalité intra-30s**: fractionner par tranches (0–10, 10–20, 20–30s) et dériver des *slopes* (diff dernier–premier, taux de croissance des volumes/holders, accélération des achats).
* **Robustesse anti-faux positifs**: pénaliser via `class_weights` + seuils plus élevés si *precision* trop basse, tout en respectant le Recall ≥ 0.75.
* **Validation temporelle**: train sur la première moitié de septembre, val sur la seconde (réalisme marché, évite fuite temporelle).
* **Features d’“intégrité créateur”**: gros `creator_balance` + `creator_sold_flag` tôt → patterns de *rug* potentiels.
* **Calibration**: isotonic / Platt (en post-fit) si la calibration améliore la recherche du seuil.
* **Sanity checks soumission**: dédupliquer `mint_token_id`, vérifier header, vérifier binaire {0,1}, compter exactement **64 208** lignes.

---

## 4) Ce qu’il te reste à brancher

* Les **chemins fichiers** (`transactions_csv`, `targets_csv`).
* La **lecture éventuelle par chunks** si ta RAM est serrée.
* Le **set d’évaluation officiel** (pour respecter 64 208 lignes exactement).
* Éventuellement, un **notebook Jupyter** qui reprend ce code en cellules + quelques graphiques de distribution.

Quand tu as les fichiers, on peut **brancher les chemins**, lancer une **première passe**, puis **itérer sur les features** et l’optim du seuil. 🚀
